In [38]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima_model import ARIMA
from sklearn.metrics import mean_absolute_error
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
from statsmodels.tsa.stattools import adfuller
from sklearn.metrics import r2_score
from statsmodels.tsa.stattools import adfuller
from pmdarima import auto_arima
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
#Ignore warnings
import warnings
import scipy
import statistics
from datetime import datetime
from platform import python_version
import statsmodels.api as sm
import matplotlib.dates as mdates
print(python_version())
warnings.filterwarnings('ignore')

3.9.7


In [32]:
df = pd.read_csv('Data\\final_burglary_data_city1.csv')
df['year month'] = [datetime.strptime(date, '%Y-%m') for date in df['year month']]
print(sorted(df['city'].unique()))

['Adur', 'Allerdale', 'Amber Valley', 'Arun', 'Ashfield', 'Ashford', 'Aylesbury Vale', 'Babergh', 'Barking and Dagenham', 'Barnet', 'Barnsley', 'Barrow-in-Furness', 'Basildon', 'Basingstoke and Deane', 'Bassetlaw', 'Bath and North East Somerset', 'Bedford', 'Bexley', 'Birmingham', 'Blaby', 'Blackburn with Darwen', 'Blackpool', 'Blaenau Gwent', 'Bolsover', 'Bolton', 'Boston', 'Bournemouth', 'Bracknell Forest', 'Bradford', 'Braintree', 'Breckland', 'Brent', 'Brentwood', 'Bridgend', 'Brighton and Hove', 'Bristol', 'Broadland', 'Bromley', 'Bromsgrove', 'Broxbourne', 'Broxtowe', 'Burnley', 'Bury', 'Caerphilly', 'Calderdale', 'Cambridge', 'Camden', 'Cannock Chase', 'Canterbury', 'Cardiff', 'Carlisle', 'Carmarthenshire', 'Castle Point', 'Central Bedfordshire', 'Ceredigion', 'Charnwood', 'Chelmsford', 'Cheltenham', 'Cherwell', 'Cheshire East', 'Cheshire West and Chester', 'Chesterfield', 'Chichester', 'Chiltern', 'Chorley', 'Christchurch', 'City of London', 'Colchester', 'Conwy', 'Copeland', '

Interesting cities:
Solihull
Adur
Blackpool
Birmingham
Brent
Cambridge
Barnet

In [ ]:
col = 'burglarys city'
n_periods = 12
city = 'Barnet'

df_city = df.loc[df['city'] == city]
# Reset index to stop expected value error

df_city_index_reset = df_city.reset_index(drop = True)

#Sort it on year/month to perform ARIMA
df_city_sorted = df_city_index_reset.set_index(df_city_index_reset['year month'])

df_city_train = df_city_sorted['2010-12':'2017-12']
adf = adfuller(df_city_train[col].dropna())

if adf[1] > 0.05:
    df_city_sorted['diff burglarys city'] = df_city_sorted[col].diff()
    col= 'diff burglarys city'
    print('WARNING!\nTime Series is non-stationary, taking the first difference \n')

#Create test partition
df_city_train = df_city_sorted['2010-12':'2017-12']
df_city_test  = df_city_sorted['2018-1':'2019-12']

plot_acf(df_city_train[col].dropna())
plot_pacf(df_city_train[col].dropna())

smodel = auto_arima(df_city_train[col].dropna(), X=None, start_p=1,
                    start_q=1, test='adf',
                    max_p=3, max_q=3, m=12,
                    start_P=0, seasonal=True,
                    d=0, D=1, trace=True,
                    error_action='ignore',
                    suppress_warnings=True,
                    stepwise=True)
print(smodel.summary())

fitted, confint = smodel.predict(n_periods=n_periods, return_conf_int=True)
index_of_fc = df_city_test.index[:n_periods]

# make series for plotting purpose
fitted_series = pd.Series(fitted, index=index_of_fc)
lower_series = pd.Series(confint[:, 0], index=index_of_fc)
upper_series = pd.Series(confint[:, 1], index=index_of_fc)

smodel.plot_diagnostics(figsize=(7, 7))
plt.show()
col = 'burglarys city'

if adf[1] > 0.05:
    for i in range(len(df_city_test[:n_periods])):
        if i == 0:
            y = df_city_train[col][-1]
        predicted_upper = y + upper_series[i]
        upper_series[i] = predicted_upper
        predicted_y = y + fitted_series[i]
        fitted_series[i] = predicted_y
        predicted_lower = y + lower_series[i]
        lower_series[i] = predicted_lower
        y=predicted_y

# Plot
plt.plot(df_city_train[col][38:].dropna())
plt.plot(df_city_test[col][:n_periods].dropna())
plt.plot(fitted_series, color='darkgreen')
plt.fill_between(lower_series.index,
                 lower_series,
                 upper_series,
                 color='k', alpha=.15)

plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
plt.gca().xaxis.set_major_locator(mdates.YearLocator())
plt.gcf().autofmt_xdate()

plt.title(f"SARIMA - Forecast amount of burglaries in {city}")
plt.show()

WARNING!
Time Series is non-stationary, taking the first difference 

Performing stepwise search to minimize aic
 ARIMA(1,0,1)(0,1,1)[12] intercept   : AIC=inf, Time=0.33 sec
 ARIMA(0,0,0)(0,1,0)[12] intercept   : AIC=827.573, Time=0.02 sec
 ARIMA(1,0,0)(1,1,0)[12] intercept   : AIC=789.603, Time=0.17 sec
 ARIMA(0,0,1)(0,1,1)[12] intercept   : AIC=inf, Time=0.22 sec
 ARIMA(0,0,0)(0,1,0)[12]             : AIC=825.575, Time=0.02 sec
 ARIMA(1,0,0)(0,1,0)[12] intercept   : AIC=812.771, Time=0.08 sec
 ARIMA(1,0,0)(2,1,0)[12] intercept   : AIC=785.554, Time=0.36 sec
 ARIMA(1,0,0)(2,1,1)[12] intercept   : AIC=inf, Time=0.53 sec
 ARIMA(1,0,0)(1,1,1)[12] intercept   : AIC=inf, Time=0.37 sec
 ARIMA(0,0,0)(2,1,0)[12] intercept   : AIC=796.257, Time=0.12 sec
 ARIMA(2,0,0)(2,1,0)[12] intercept   : AIC=776.739, Time=0.37 sec
 ARIMA(2,0,0)(1,1,0)[12] intercept   : AIC=777.883, Time=0.26 sec
 ARIMA(2,0,0)(2,1,1)[12] intercept   : AIC=inf, Time=0.63 sec
 ARIMA(2,0,0)(1,1,1)[12] intercept   : AIC=inf, T

In [ ]:
mae_mean = mean_absolute_error(df_city_test[col][:n_periods].dropna(), [df_city_train[col].mean()]*12)
mae_model = mean_absolute_error(df_city_test[col][:n_periods].dropna(), fitted_series)
diff_mae = (mae_model - mae_mean)/mae_mean
print('MAE avarage amount of burglaries:',mae_mean)
print('MAE SARIMAX model:', mae_model)
print(diff_mae)

In [72]:
diff_mae_lst = []
mae_lst = []
for city in sorted(df['city'].unique()):
    try:
        #print(city)
        col = 'burglarys city'
        n_periods = 12

        df_city = df.loc[df['city'] == city]
        # Reset index to stop expected value error

        df_city_index_reset = df_city.reset_index(drop = True)

        #Sort it on year/month to perform ARIMA
        df_city_sorted = df_city_index_reset.set_index(df_city_index_reset['year month'])

        df_city_train = df_city_sorted['2010-12':'2017-12']
        adf = adfuller(df_city_train[col].dropna())

        if adf[1] >0.05:
            df_city_sorted['diff burglarys city'] = df_city_sorted[col].diff()
            col= 'diff burglarys city'
            #print('WARNING!\nTime Series is non-stationary, taking the first difference \n')

        #Create test partition
        df_city_train = df_city_sorted['2010-12':'2017-12']
        df_city_test  = df_city_sorted['2018-1':'2019-12']


        smodel = auto_arima(df_city_train[col].dropna(), start_p=1, start_q=1,
                            test='adf',
                            max_p=3, max_q=3, m=12,
                            start_P=0, seasonal=False,
                            d=0, D=1, trace=False,
                            error_action='ignore',
                            suppress_warnings=True,
                            stepwise=True)


        fitted, confint = smodel.predict(n_periods=n_periods, return_conf_int=True)
        index_of_fc = df_city_test.index[:n_periods]

        # make series for plotting purpose
        fitted_series = pd.Series(fitted, index=index_of_fc)
        lower_series = pd.Series(confint[:, 0], index=index_of_fc)
        upper_series = pd.Series(confint[:, 1], index=index_of_fc)

        col = 'burglarys city'
        if adf[1] > 0.05:
            for i in range(len(df_city_test[:n_periods])):
                if i == 0:
                    y = df_city_train[col][-1]
                predicted_upper = y + upper_series[i]
                upper_series[i] = predicted_upper
                predicted_y = y + fitted_series[i]
                fitted_series[i] = predicted_y
                predicted_lower = y + lower_series[i]
                lower_series[i] = predicted_lower
                y=predicted_y

        mae_mean = mean_absolute_error(df_city_test[col][:n_periods].dropna(), [df_city_train[col].mean()]*12)
        mae_model = mean_absolute_error(df_city_test[col][:n_periods].dropna(), fitted_series)
        diff_mae = (mae_model - mae_mean)/mae_mean
        diff_mae_lst.append(diff_mae)
        info = [city, 'mean:',mae_mean,'model:', mae_model, diff_mae]
        print(info, '\n')
        mae_lst.append(info)
        print(statistics.mean(diff_mae_lst), '\n')
    except:
        continue


['Adur', 'mean:', 9.862745098039218, 'model:', 7.71702707353052, -0.2175578911529693] 

-0.2175578911529693 

['Allerdale', 'mean:', 5.3960784313725485, 'model:', 5.562406157803603, 0.03082381558133646] 

-0.09336703778581643 

['Amber Valley', 'mean:', 15.490196078431373, 'model:', 6.720907808854897, -0.5661186098081015] 

-0.2509508951265781 

['Arun', 'mean:', 8.039215686274504, 'model:', 6.467962649033924, -0.19544854853480398] 

-0.23707530847863456 

['Ashfield', 'mean:', 14.59019607843137, 'model:', 14.177078282783691, -0.028314752826275547] 

-0.19532319734816278 

['Ashford', 'mean:', 18.590196078431372, 'model:', 17.60097890092371, -0.05321176674706335] 

-0.1716379589146462 

['Aylesbury Vale', 'mean:', 15.615686274509804, 'model:', 15.27006500652992, -0.022132954127290355] 

-0.1502801010878811 

['Babergh', 'mean:', 10.742156862745098, 'model:', 10.634321555851367, -0.010038515381181534] 

-0.13274990287454363 

['Barking and Dagenham', 'mean:', 31.316666666666663, 'model:

In [74]:
print(statistics.mean(diff_mae_lst))
mae_lst

-0.007553291463374636


[['Adur',
  'mean:',
  9.862745098039218,
  'model:',
  7.71702707353052,
  -0.2175578911529693],
 ['Allerdale',
  'mean:',
  5.3960784313725485,
  'model:',
  5.562406157803603,
  0.03082381558133646],
 ['Amber Valley',
  'mean:',
  15.490196078431373,
  'model:',
  6.720907808854897,
  -0.5661186098081015],
 ['Arun',
  'mean:',
  8.039215686274504,
  'model:',
  6.467962649033924,
  -0.19544854853480398],
 ['Ashfield',
  'mean:',
  14.59019607843137,
  'model:',
  14.177078282783691,
  -0.028314752826275547],
 ['Ashford',
  'mean:',
  18.590196078431372,
  'model:',
  17.60097890092371,
  -0.05321176674706335],
 ['Aylesbury Vale',
  'mean:',
  15.615686274509804,
  'model:',
  15.27006500652992,
  -0.022132954127290355],
 ['Babergh',
  'mean:',
  10.742156862745098,
  'model:',
  10.634321555851367,
  -0.010038515381181534],
 ['Barking and Dagenham',
  'mean:',
  31.316666666666663,
  'model:',
  25.583333333333332,
  -0.18307610431080357],
 ['Barnet',
  'mean:',
  44.06862745098041,

In [75]:
print(statistics.median(diff_mae_lst))

-0.01411242327212346


In [137]:
all_adf = []
for city in sorted(df['city'].unique()):
    df_city = df.loc[df['city'] == city]
    # Reset index to stop expected value error

    df_city_index_reset = df_city.reset_index(drop = True)

    #Sort it on year/month to perform ARIMA
    df_city_sorted = df_city_index_reset.set_index(df_city_index_reset['year month'])

    #Create test partition
    df_city_train = df_city_sorted['2010-12':'2017-12']
    df_city_test  = df_city_sorted['2017-12':'2019-12']
    adf = adfuller(df_city_train[col].dropna())
    all_adf.append([adf[1], city])

In [129]:
count = 0
for p_value in all_adf:
    if p_value[0] > 0.05:
        print(p_value)
        count+=1
print(f'percentage of cities with a non-stationary time series: {(count/len(all_adf))*100}')

[0.12379602018514313, 'Amber Valley']
[0.8197394408901681, 'Barking and Dagenham']
[0.6690847719840085, 'Barnet']
[0.16416670590697602, 'Barnsley']
[0.3121104569507079, 'Birmingham']
[0.6118531835209174, 'Bolsover']
[0.37463663921148715, 'Bracknell Forest']
[0.28466287826409564, 'Bradford']
[0.761108449751251, 'Brent']
[0.3093753497543549, 'Bridgend']
[0.17750002482021277, 'Brighton and Hove']
[0.78900140503725, 'Bromley']
[0.8802255884924897, 'Bromsgrove']
[0.1714308836400229, 'Broxtowe']
[0.2710978008103583, 'Caerphilly']
[0.10586576036332707, 'Cambridge']
[0.09057953937425195, 'Cardiff']
[0.560231983025205, 'Castle Point']
[0.5592115643575387, 'Cherwell']
[0.3573064629831805, 'Cheshire West and Chester']
[0.30474988069151976, 'Chichester']
[0.3484844670132945, 'Chiltern']
[0.0655425402195777, 'Corby']
[0.6099586148989727, 'Cornwall']
[0.16892929410676188, 'Coventry']
[0.6121622687091509, 'Crawley']
[0.4786515087257871, 'Croydon']
[0.12146794734915523, 'Darlington']
[0.14266232967891

In [64]:
df[df['city']== "Isles of Scilly"]

,Unnamed: 0,Longitude,Latitude,month,year,median house price,burglarys city,burglarys lsoa,city,police district,year month
2576,2576,-6.340993,49.956769,7.0,11.0,370000.0,1.0,1.0,Isles of Scilly,Devon & Cornwall Police,2011-07-01
2924,2924,-6.340993,49.956769,8.0,11.0,370000.0,1.0,1.0,Isles of Scilly,Devon & Cornwall Police,2011-08-01
3619,3619,-6.311885,49.915754,10.0,11.0,370000.0,1.0,1.0,Isles of Scilly,Devon & Cornwall Police,2011-10-01
4661,4661,-6.321281,49.913154,1.0,12.0,370000.0,1.0,1.0,Isles of Scilly,Devon & Cornwall Police,2012-01-01
5703,5703,-6.299174,49.912763,4.0,12.0,370000.0,1.0,1.0,Isles of Scilly,Devon & Cornwall Police,2012-04-01
9868,9868,-6.315416,49.914885,4.0,13.0,370000.0,1.0,1.0,Isles of Scilly,Devon & Cornwall Police,2013-04-01
10563,10563,-6.315029,49.914296,6.0,13.0,370000.0,2.0,2.0,Isles of Scilly,Devon & Cornwall Police,2013-06-01
10911,10911,-6.298549,49.913416,7.0,13.0,370000.0,1.0,1.0,Isles of Scilly,Devon & Cornwall Police,2013-07-01
11606,11606,-6.305400,49.917102,9.0,13.0,370000.0,1.0,1.0,Isles of Scilly,Devon & Cornwall Police,2013-09-01
12301,12301,-6.336986,49.959358,11.0,13.0,370000.0,2.0,2.0,Isles of Scilly,Devon & Cornwall Police,2013-11-01
